In [1]:
import os
os.chdir('/Volumes/Users/ly_k1/Documents/IndependentStudy/')

# Content of the Project

1. Set Up
2. Web-scraping
3. Exploratory Data Analysis

# Scrape Book Genres

In [2]:
import pandas as pd

metadata_df = pd.read_csv("data/metadata.csv", index_col = 0)
metadata_df.tail(3)

,item_id,url,title,authors,lang,img,year,description
9371,2021847,https://www.goodreads.com/book/show/25104.The_...,The Wake (The Sandman #10),Neil Gaiman,eng,https://images.gr-assets.com/books/1339922545m...,1999,"When a Dream ends, there is only one thing lef..."
9372,645677,https://www.goodreads.com/book/show/659600.Ann...,Annie's Song,Catherine Anderson,eng,https://images.gr-assets.com/books/1325836635m...,1996,Annie Trimble lives in a solitary world that n...
9373,963076,https://www.goodreads.com/book/show/218659.Eve...,Everything That Rises Must Converge: Stories,Flannery O'Connor,eng,https://images.gr-assets.com/books/1412859621m...,1996,Librarian's Note: For an alternate hardcover e...


In [3]:
metadata_df['url'][0]

'https://www.goodreads.com/book/show/11870085-the-fault-in-our-stars'

In [4]:
import json
import time
import helper

done = False
i = 0

while not done:
    row_dict = {}
    for row_index, row_series in metadata_df[['item_id','url']][i:i+1000].iterrows():
        row_dict[row_series[0]] = helper.get_genres(row_series[1])
        
    file_name = 'data/genres-data-full/genres-' + str(int(i/1000)) + '.json'
    
    with open(file_name,'w') as output_file:
        json.dump(row_dict, output_file)
        
    print(i, "/9374 items completed")
    i+=1000
    time.sleep(40)
    
    if i == 9000:
        for row_index, row_series in metadata_df[['item_id','url']][i:].iterrows():
            row_dict[row_series[0]] = get_genres(row_series[1])
        file_name = 'data/genres-data-full/genres-' + str(int(i/1000)) + '.json'
        with open(file_name,'w') as output_file:
            json.dump(row_dict, output_file)
        print("9374/9374 items completed")
        done = True

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


FeatureNotFound: Couldn't find a tree builder with the features you requested: lxml. Do you need to install a parser library?

# Scrape weight of book genres

In [ ]:
import json

file_names = ['data/genres-data-full/genres-' + str(i) + '.json' for i in range(0,10)]
file_names[0]

'data/genres-data-full/genres-0.json'

In [ ]:
all_dols = {}
for i in range(0,10):
    with open(file_names[i],'r') as input_file:
        dol = json.load(input_file)
        all_dols.update(dol)

In [ ]:
import time
import helper

book_dod = {}
i = 0

for key, value in all_dols.items():
    
    book_dod[key] = helper.get_weighted_genre(key, value)
    
    if i % 250 == 0:
        print(str(i) + "/9374 books are done")
        time.sleep(5)
    i+=1

In [ ]:
import json

with open('weighted_genre.json','w') as output_file:
    json.dump(book_dod, output_file)

NameError: name 'book_dod' is not defined

# Scrape Missing Data

In [ ]:
import json

with open('data/weighted_genre.json') as input_file:
    wg_json = json.load(input_file)

In [ ]:
import pandas as pd

wg_df = pd.DataFrame(wg_json).T
wg_df.head(5)

,young-adult,romance,fiction,contemporary,realistic-fiction,teen,coming-of-age,fantasy,dystopia,science-fiction,...,romanovs,new-adult-romance,womens,agriculture,sustainability,trains,wonder-woman,computer-science,rwanda,utopia
16827462,"22,752","14,714","12,425","7,839","2,537","1,335","1,165",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2792775,"33,322","4,612","19,505",NaN,NaN,NaN,NaN,"16,141","13,952","11,551",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8812783,"26,507","3,340","13,850",NaN,NaN,NaN,NaN,"11,423","11,500","8,525",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41107568,NaN,NaN,"11,173","1,688",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6171458,"28,081","3,701","14,806",NaN,NaN,NaN,NaN,"12,384","12,042","9,075",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Data Cleaning

In [ ]:
def fix_numbers(num):
    if isinstance(num, str):
        num = num.replace(',','')
        return int(num)
    else:
        return 0

In [ ]:
wg_df = wg_df.applymap(fix_numbers)

In [ ]:
missing_nans = list(wg_df[wg_df.sum(axis = 1)== 0].index)

In [ ]:
missing_nans

['52959357',
 '15677347',
 '2247074',
 '40159220',
 '17054802',
 '56065453',
 '51979220',
 '44033312']

In [ ]:
import pandas as pd

metadata_df = pd.read_csv('data/metadata.csv', index_col = 0)
metadata_df['item_id'] = metadata_df['item_id'].astype('str')

In [ ]:
missing_df = metadata_df[metadata_df['item_id'].isin(missing_nans)]
missing_df.head(3)

,item_id,url,title,authors,lang,img,year,description
2863,52959357,https://www.goodreads.com/book/show/34273236-l...,Little Fires Everywhere,Celeste Ng,eng,https://images.gr-assets.com/books/1490351351m...,2017,"In Shaker Heights, a placid, progressive subur..."
4959,15677347,https://www.goodreads.com/book/show/10765906-t...,"The Shadow Reader (Shadow Reader, #1)",Sandy Williams,en-US,https://images.gr-assets.com/books/1316820446m...,2011,There can only be one allegiance.\nIt's her ti...
5372,2247074,https://www.goodreads.com/book/show/32085.All_...,All Creatures Great and Small (All Creatures G...,James Herriot,eng,https://s.gr-assets.com/assets/nophoto/book/11...,0,The classic multimillion copy bestseller\nDelv...


In [ ]:
import helper
dol = {}

i = 0
for row_idx, row_series in missing_df.iterrows():
    dol[row_series[0]] = helper.get_genres(row_series[1])
    
    i+=1
    if i % 50 == 0:
        print(str(i) + '/259 books done')
print('259/259 books done')

259/259 books done


In [ ]:
import numpy as np 

for key,value in dol.items():
    if value == np.nan:
        print(value)

In [ ]:
import helper

missing_dod = {}
i = 0 

for key, value in dol.items():
    
    missing_dod[key] = helper.get_weighted_genre(str(key), value)
    
    if i % 50 == 0:
        print(str(i) + "/259 books are done")
    i+=1
print('259/259 books done')

/Volumes/Users/ly_k1/Documents/IndependentStudy/scraping_lib.py:27: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 27 of the file /Volumes/Users/ly_k1/Documents/IndependentStudy/scraping_lib.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  bs = BeautifulSoup(r.text)


0/259 books are done
259/259 books done


In [ ]:
for key, value in missing_dod.items():
    wg_json[key] = value

In [ ]:
with open('weighted_genre.json','w') as output_file:
    json.dump(wg_json, output_file)

## Scrape Missing Release Year

In [ ]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import re

user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.2 Safari/605.1.15'
headers = {'User-Agent': user_agent}

def get_year(item_id, url):
    times = 0
    while times <= 10:
        try:
            r = requests.get(url, headers = headers)
            bs = BeautifulSoup(r.text)
            target = bs.find('div', class_ = 'uitext darkGreyText', id = 'details').text
            pattern = r'Published(?:.|\n)+(\d{4})(?:.|\n)+More Details'
            year = int(re.findall(pattern, target)[0])
            return [item_id, year]
        except:
            times+=1
            continue
    return [np.nan, np.nan]

In [ ]:
import pandas as pd
metadata_df = pd.read_csv('data/metadata_with_genres.csv', index_col = 0)

In [ ]:
item_id_list = list(metadata_df[metadata_df['year'] == 0]['item_id'])
url_list = list(metadata_df[metadata_df['year'] == 0]['url'])

In [ ]:
row_list = []
for i in range(len(url_list)):
    row_list.append(get_year(item_id_list[i], url_list[i]))

In [ ]:
df = pd.DataFrame(row_list, columns=['item_id', 'year'])
df.to_csv('data/missing-year.csv')